# Compustat에서 필요한 firm-lv 변수 뽑기

### 1. gvkey, year selection
- app_cit에 있는 기업만 고르기

### 2. 단순 변수 선택 & 제작
- ROA (ebit / at)
- 이익률 (ebit / revt)
- R&D intensity (xrd / revt)

### 3. market concentration
- 이건 전체 compustat 데이터로 비교 필요
- sic 코드 같은 애들 싹 다 모은 후 구하기

### 1. selection

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps')

In [2]:
raw = pd.read_csv('db_table/compustat_05-12.csv')

bio = pd.read_csv('appcit_bio.csv')
bio = bio.rename(columns = {'year':'fyear'})
bio.drop('app_cit', axis=1, inplace=True)

ee = pd.read_csv('appcit_EE&IT.csv')
ee = ee.rename(columns = {'year':'fyear'})
ee.drop('app_cit', axis=1, inplace=True)

len(bio), len(ee)

(1445, 2665)

In [3]:
bio = bio.merge(raw,how='left',on=['gvkey','fyear'])
ee = ee.merge(raw,how='left',on=['gvkey','fyear'])

len(bio), len(ee)

(1445, 2665)

In [7]:
bio.to_csv('compu_bio.csv', index=False)
ee.to_csv('compu_EE&IT.csv',index=False)

### 2. simple feature 제작
- ROA : ebit, at 없는 애들 못 구함
- 이익률 : ebit, revt 없는 애들 못 구함
- R&D intensity : xrd, revt 없는 애들 못 구함  
=> dropna on [ebit, at, revt, (xrd)]

__2.1 drop na__

In [4]:
list(bio)

['gvkey',
 'fyear',
 'conm',
 'consol',
 'datadate',
 'fic',
 'curcd',
 'act',
 'at',
 'ebit',
 'emp',
 'fatb',
 'fate',
 'fatl',
 'fatp',
 'fopo',
 'fsrco',
 'fsrcopo',
 'fsrcopt',
 'fsrct',
 'fuseo',
 'fuset',
 'ppegt',
 'ppent',
 'revt',
 'teq',
 'xlr',
 'xrd',
 'xstfws',
 'ni',
 'sic']

In [5]:
colnm = list(bio)
print colnm.index('fatb'), colnm.index('fuset')

11 21


In [6]:
bio.drop(colnm[11:22], axis=1, inplace=True)
ee.drop(colnm[11:22], axis=1, inplace=True)

In [7]:
bio1 = bio.dropna(axis=0, how='any',subset=['at','revt','ebit'])
bio2 = bio.dropna(axis=0, how='any',subset=['at','revt','ebit','xrd'])

print len(bio), len(bio1), len(bio2)

1445 1341 1300


In [8]:
ee1 = ee.dropna(axis=0, how='any',subset=['at','revt','ebit'])
ee2 = ee.dropna(axis=0, how='any',subset=['at','revt','ebit','xrd'])

print len(ee), len(ee1), len(ee2)

2665 2542 2411


In [9]:
bio2.to_csv('compu_bio_na.csv', index=False)
ee2.to_csv('compu_EE&IT_na.csv',index=False)

__2.2 변수 제작__

In [2]:
bio = pd.read_csv('compu_bio_na.csv')
ee = pd.read_csv('compu_EE&IT_na.csv')

bio['roa'] = bio['ebit'] / bio['at']
bio['margin'] = bio['ebit'] / bio['revt']
bio['rdi'] = bio['xrd'] / bio['revt']

ee['roa'] = ee['ebit'] / ee['at']
ee['margin'] = ee['ebit'] / ee['revt']
ee['rdi'] = ee['xrd'] / ee['revt']

In [5]:
bio.to_csv('compu_bio_temp.csv', index=False)
ee.to_csv('compu_EE&IT_temp.csv',index=False)

### 3. market concentration
- 4digit, 3digit lv ㄱㄱ
- compu_temp 기업 gvkey, year, sic 뽑기
- 진짜 compustat에서 sic 매칭, gvkey, year, sic, revt 뽑기
- groupby sic, year sum()
- sic, year로 compu_temp에 join, 각 기업별 concentration 구함

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps')

__4digit 후 3digit으로 넘어가자__

In [2]:
bio = pd.read_csv('compu_bio_temp.csv')
ee = pd.read_csv('compu_EE&IT_temp.csv')

bio = bio.loc[:,['fyear','sic']]
bio.drop_duplicates(inplace=True)

ee = ee.loc[:,['fyear','sic']]
ee.drop_duplicates(inplace=True)

len(bio), len(ee)

(234, 558)

In [3]:
comp = pd.read_csv('db_table/compustat_05-12.csv')
comp = comp.loc[:,['gvkey','fyear','revt','sic']]
comp.drop_duplicates(inplace=True)

bio = bio.merge(comp,how='left',on=['sic','fyear'])
ee = ee.merge(comp,how='left',on=['sic','fyear'])

len(bio), len(ee)

(32953, 81963)

In [4]:
del bio['gvkey']
bio = bio.groupby(['sic','fyear'], as_index=False).sum()

del ee['gvkey']
ee = ee.groupby(['sic','fyear'], as_index=False).sum()

len(bio), len(ee)

(234, 558)

In [5]:
bio['sic'] = bio['sic'].apply(int,1)
ee['sic'] = ee['sic'].apply(int,1)

bio=bio.rename(columns = {'revt':'revt_sic4'})
ee=ee.rename(columns = {'revt':'revt_sic4'})

ee.iloc[0:10,:]

,sic,fyear,revt_sic4
0,1381,2005,3.724061e+04
1,1381,2006,1.405922e+06
2,1381,2007,2.802875e+06
3,1381,2008,3.811498e+06
4,1381,2009,4.170336e+06
5,1381,2010,7.651192e+06
6,1381,2011,9.305952e+06
7,1381,2012,1.207669e+07
8,1389,2005,3.325716e+06
9,1389,2006,4.601941e+06


In [6]:
len(bio['sic'].unique()), len(ee['sic'].unique())

(41, 81)

In [7]:
bio.to_csv('sic_4_bio.csv',index=False)
ee.to_csv('sic_4_ee.csv',index=False)

__바탕으로 3digit ㄱㄱ__

In [8]:
def to_3d(sic):
    sic = str(sic)[:-1]
    return int(sic)

bio['sic_3'] = bio['sic'].apply(to_3d,1)
ee['sic_3'] = ee['sic'].apply(to_3d,1)

bio3 = bio.drop('sic',axis=1)
ee3 = ee.drop('sic',axis=1)

ee.iloc[0:10,:]

,sic,fyear,revt_sic4,sic_3
0,1381,2005,3.724061e+04,138
1,1381,2006,1.405922e+06,138
2,1381,2007,2.802875e+06,138
3,1381,2008,3.811498e+06,138
4,1381,2009,4.170336e+06,138
5,1381,2010,7.651192e+06,138
6,1381,2011,9.305952e+06,138
7,1381,2012,1.207669e+07,138
8,1389,2005,3.325716e+06,138
9,1389,2006,4.601941e+06,138


In [9]:
bio3 = bio3.groupby(['sic_3','fyear'], as_index=False).sum()
ee3 = ee3.groupby(['sic_3','fyear'], as_index=False).sum()

bio3 = bio3.rename(columns = {'revt_sic4':'revt_sic3'})
ee3 = ee3.rename(columns = {'revt_sic4':'revt_sic3'})

len(bio3['sic_3'].unique()), len(ee3['sic_3'].unique()), 

(30, 53)

In [10]:
bio3.to_csv('sic_3_bio.csv',index=False)
ee3.to_csv('sic_3_ee.csv',index=False)

__concetration 계산__

In [11]:
bio4 = bio
ee4 = ee

bio = pd.read_csv('compu_bio_temp.csv')
ee = pd.read_csv('compu_EE&IT_temp.csv')

bio = bio.merge(bio4,how='left',on=['sic','fyear'])
ee = ee.merge(ee4,how='left',on=['sic','fyear'])

bio = bio.merge(bio3,how='left',on=['sic_3','fyear'])
ee = ee.merge(ee3,how='left',on=['sic_3','fyear'])

In [14]:
bio['mkt_4'] = bio['revt'] / bio['revt_sic4']
bio['mkt_3'] = bio['revt'] / bio['revt_sic3']

ee['mkt_4'] = ee['revt'] / ee['revt_sic4']
ee['mkt_3'] = ee['revt'] / ee['revt_sic3']

In [17]:
bio.to_csv('feature/compu_bio.csv',index=False)
ee.to_csv('feature/compu_EE&IT.csv',index=False)